# Setup for google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import sys
%cd "/content/drive/MyDrive/ColabNotebooks/Audio-ex3"
sys.path.append("/content/drive/MyDrive/ColabNotebooks/Audio-ex3")

In [ ]:
!git pull

In [ ]:
!pip install -r requirements.txt
%cd src

---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

True

---
# Loading Data

In [1]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
dm = loader.AudioDataModule()
dm.setup("fit")

---
# Logger

In [2]:
from dotenv import load_dotenv
import os

load_dotenv('./../.env')
API_TOKEN = os.environ.get("API_TOKEN")
PROJECT_NAME = os.environ.get("PROJECT_NAME")

In [12]:
from lightning.pytorch.loggers import NeptuneLogger

logger_config = {
    "api_key": API_TOKEN,
    "project": PROJECT_NAME,
    "log_model_checkpoints": True,
    "tags": ["LSTM++", "Linear"]
}

---
# Create the classifier and train it

In [ ]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

neptune_logger = NeptuneLogger(**logger_config)
config = classifier.default_config

trainer = classifier.train_func(config=config, dm=dm, logger=neptune_logger, num_epochs=50)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-183


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type        | Params | Mode 
-----------------------------------------------------
0 | batch_norm   | BatchNorm1d | 0      | train
1 | loss         | CTCLoss     | 0      | train
2 | conv         | Sequential  | 14.8 K | train
3 | bi_rnn       | GRU         | 6.3 M  | train
4 | linear_final | Linear      | 54.8 K | train
-----------------------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
25.320    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

---
# Test the classifier

# prepre the test data

In [7]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
dm_test = loader.AudioDataModule()
dm_test.setup("test")

# Trainer

In [8]:
from lightning.pytorch.loggers import NeptuneLogger

logger_config = {
    "api_key": API_TOKEN,
    "project": PROJECT_NAME,
    "log_model_checkpoints": False,
    "tags": ["test"]
}
neptune_logger_test = NeptuneLogger(**logger_config)

In [9]:
import lightning.pytorch as pl
trainer_test = pl.Trainer(
        devices="auto",
        accelerator="auto",
        logger=neptune_logger_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## Load the model

In [10]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

model_test = classifier.DigitClassifier.load_from_checkpoint(".neptune/AUD-181/AUD-181/checkpoints/epoch=49-step=20000.ckpt", n_feature=loader.MFCC_FEATURES)

## Test the model

In [11]:
trainer_test.test(model_test, datamodule=dm_test)
neptune_logger_test.run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-182


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Documents\GitHub\Audio-ex3\.venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test_avg_accuracy          0.9111328125
      test_avg_loss         0.5979229211807251
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 4 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 4 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-182/metadata


---
# Validation

In [ ]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
importlib.reload(importlib.import_module("model_tuner"))
import model_tuner

model_tuner.tune_classifier(logger_config=logger_config, dm=dm)